In [ ]:
import os
import numpy as np
import git
import mlflow
import torch
import torch.nn as nn
import math
from torch.utils.data import TensorDataset, DataLoader

In [15]:
# --- 1. Метрики и Scoring Functions ---

def calculate_metrics(y_true, y_pred):
    """
    Вычисляет расширенный набор метрик для RUL.
    y_true, y_pred: torch.Tensor или numpy array
    """
    if isinstance(y_true, torch.Tensor):
        y_true = y_true.detach().cpu().numpy()
    if isinstance(y_pred, torch.Tensor):
        y_pred = y_pred.detach().cpu().numpy()
        
    y_true = y_true.flatten()
    y_pred = y_pred.flatten()
    
    # Разница
    d = y_pred - y_true
    
    # 1. MAE
    mae = np.mean(np.abs(d))
    
    # 2. RMSE
    rmse = np.sqrt(np.mean(d**2))
    
    # 3. MAPE
    mape = np.mean(np.abs((y_true - y_pred) / np.maximum(np.abs(y_true), 1.0))) * 100

    # 4. PHM08 Score (NASA Scoring Function) [web:PHM08_Challenge]
    # Функция асимметрична: ранние предсказания (d < 0) штрафуются меньше, чем поздние (d > 0)
    # Формула: sum(exp(-d/13) - 1 если d < 0, иначе exp(d/10) - 1)
    # *Внимание: в условии было указано score = sum(...) / n. Обычно в PHM08 используют просто sum,
    # но для сопоставимости метрик лучше использовать среднее (mean) или следовать условию задачи.
    # Здесь реализуем согласно вашему ТЗ: делим на n.
    n = len(d)
    scores = np.where(d < 0, np.exp(-d/13) - 1, np.exp(d/10) - 1)
    phm08_score = np.sum(scores) / n
    
    return {"mae": mae, "rmse": rmse, "mape": mape, "phm08_score": phm08_score}

In [ ]:
# --- 2. Адаптивная модель (Transfer Learning Ready) ---

class PositionalEncoding(nn.Module):
    """
    Добавляет информацию о позиции в последовательности.
    Стандартная реализация из PyTorch туториалов.
    """
    def __init__(self, d_model, max_len=5000, dropout=0.1):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        # Создаем матрицу позиций
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        
        # Добавляем размерность батча: (1, max_len, d_model)
        self.register_buffer('pe', pe.unsqueeze(0))

    def forward(self, x):
        # x: (Batch, Seq_Len, Embed_Dim)
        # Добавляем позицию к входным данным (срезаем pe под длину текущей последовательности)
        x = x + self.pe[:, :x.size(1), :]
        return self.dropout(x)

class AdaptiveTransformerModel(nn.Module):
    def __init__(self, input_dim, embed_dim=64, num_heads=4, ff_dim=128, num_layers=2, output_dim=1, dropout=0.1):
        super(AdaptiveTransformerModel, self).__init__()
        
        # === 1. АДАПТИВНЫЙ ВХОДНОЙ СЛОЙ (ADAPTER) ===
        # Проецируем N фичей в размерность модели (embed_dim).
        # Это сменная часть для Transfer Learning.
        self.input_adapter = nn.Linear(input_dim, embed_dim)
        
        # === 2. BACKBONE (ЯДРО: TRANSFORMER) ===
        
        # а) Кодирование позиции
        self.pos_encoder = PositionalEncoding(embed_dim, dropout=dropout)
        
        # б) Блок Энкодера
        # batch_first=True означает вход (Batch, Seq, Feature)
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=embed_dim, 
            nhead=num_heads, 
            dim_feedforward=ff_dim, 
            dropout=dropout,
            batch_first=True
        )
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        
        # === 3. ГОЛОВА (HEAD) ===
        # Принимает усредненный вектор (Global Average Pooling)
        self.fc = nn.Linear(embed_dim, output_dim)

    def forward(self, x):
        # x: (batch_size, seq_len, input_dim)
        
        # 1. Адаптация + Масштабирование
        # В трансформерах принято умножать эмбеддинги на sqrt(d_model) перед позицией
        x = self.input_adapter(x) * math.sqrt(self.input_adapter.out_features)
        
        # 2. Добавление информации о порядке времени
        x = self.pos_encoder(x)
        # Shape: (batch_size, seq_len, embed_dim)
        
        # 3. Проход через Transformer Encoder
        x = self.transformer_encoder(x)
        # Shape: (batch_size, seq_len, embed_dim)
        
        # 4. Global Average Pooling 1D
        # Усредняем по временной оси (dim=1)
        # Было: (Batch, Seq, Emb) -> Стало: (Batch, Emb)
        x = x.mean(dim=1) 
        
        # 5. Финальный прогноз
        final_output = self.fc(x)
        return final_output


In [ ]:
def adapt_for_customer_data(
    backbone_path,
    new_input_dim,
    embed_dim=64,
    num_heads=4,
    ff_dim=128,
    num_layers=2,
    dropout=0.1,
    output_dim=1,
    freeze_backbone=True,
):
    """
    Создает модель для новых данных заказчика, загружая предобученный Transformer-backbone.
    """
    # 1. Инициализируем модель с НОВОЙ размерностью входа
    new_model = AdaptiveTransformerModel(
        input_dim=new_input_dim,  # Например, 15 сенсоров вместо 20
        embed_dim=embed_dim,
        num_heads=num_heads,
        ff_dim=ff_dim,
        num_layers=num_layers,
        output_dim=output_dim,
        dropout=dropout,
    )

    # 2. Загружаем веса Backbone
    # strict=False позволяет игнорировать несовпадение ключей (т.к. input_adapter у нас новый)
    pretrained_dict = torch.load(backbone_path, map_location="cpu")
    model_dict = new_model.state_dict()

    # Фильтруем, чтобы загрузить только backbone
    pretrained_dict = {k: v for k, v in pretrained_dict.items() if k in model_dict and "input_adapter" not in k}

    # Обновляем веса
    model_dict.update(pretrained_dict)
    new_model.load_state_dict(model_dict, strict=False)

    # 3. Замораживаем Backbone (опционально, если хотим учить только адаптер)
    if freeze_backbone:
        for name, param in new_model.named_parameters():
            if "input_adapter" not in name:
                param.requires_grad = False

    print(
        f"Model adapted for input_dim={new_input_dim}. Backbone weights loaded"
        f"{' and frozen' if freeze_backbone else ''}."
    )
    return new_model

In [ ]:
# Указываем MLflow, куда отправлять данные
mlflow.set_tracking_uri("http://213.21.252.250:5000")

# Задаем имя эксперимента
mlflow.set_experiment("Transformer_TransferLearning")

# --- Получаем хеш коммита Git ---
try:
    repo = git.Repo(search_parent_directories=True)
    git_commit_hash = repo.head.object.hexsha
except Exception as e:
    git_commit_hash = "N/A"  # На случай, если скрипт запущен не из Git-репозитория
    print(f"Warning: Could not get git commit hash. {e}")

print(f"Current Git Commit Hash: {git_commit_hash}")

# --- Параметры, которые нужно логировать ---
# Параметры из скрипта нарезки данных (sample_creator)
data_params = {
    "window_size": 100,
    "step": 1,
    "sampling_rate": 10,
}

# Гиперпараметры модели (Transformer)
model_params = {
    "epochs": 7,
    "batch_size": 128,
    "validation_split": 0.2,
    "optimizer": "adam",
    "loss": "mean_squared_error",
    "lr": 0.002,
    "embed_dim": 64,
    "num_heads": 4,
    "ff_dim": 128,
    "num_layers": 2,
    "dropout": 0.1,
}

Current Git Commit Hash: 10e7174a76b97d41dbe6a5b23052a7fc80aa8ee5


In [19]:
def load_and_merge_data(npz_units):
      sample_array_lst = []
      label_array_lst = []
      for npz_unit in npz_units:
        loaded = np.load(npz_unit)
        sample_array_lst.append(loaded['sample'])
        label_array_lst.append(loaded['label'])
      sample_array = np.dstack(sample_array_lst)
      label_array = np.concatenate(label_array_lst)
      sample_array = sample_array.transpose(2, 0, 1)
      return sample_array, label_array

processed_dir = '../data/processed/'

# Собираем пути к файлам для train и test
train_files = [os.path.join(processed_dir, f) for f in os.listdir(processed_dir) if f.startswith(('Unit2_', 'Unit5_', 'Unit10_', 'Unit16_', 'Unit18_', 'Unit20_'))]
test_files = [os.path.join(processed_dir, f) for f in os.listdir(processed_dir) if f.startswith(('Unit11_', 'Unit14_', 'Unit15_'))]
print(train_files)

# Загружаем данные
X_train, y_train = load_and_merge_data(train_files)
X_test, y_test = load_and_merge_data(test_files)

# Определяем форму входных данных из X_train
n_timesteps, n_features = X_train.shape[1], X_train.shape[2]

print('Размер обучающей выборки (X):', X_train.shape)
print('Размер обучающей выборки (y):', y_train.shape)
print('Размер тестовой выборки (X):', X_test.shape)
print('Размер тестовой выборки (y):', y_test.shape)

['../data/processed/Unit5_win100_str1_smp10.npz', '../data/processed/Unit16_win100_str1_smp10.npz', '../data/processed/Unit2_win100_str1_smp10.npz', '../data/processed/Unit20_win100_str1_smp10.npz', '../data/processed/Unit18_win100_str1_smp10.npz', '../data/processed/Unit10_win100_str1_smp10.npz']
Размер обучающей выборки (X): (525751, 100, 20)
Размер обучающей выборки (y): (525751,)
Размер тестовой выборки (X): (125077, 100, 20)
Размер тестовой выборки (y): (125077,)


In [ ]:
with mlflow.start_run():
    print("Starting MLflow run...")

    # --- Логируем параметры ---
    mlflow.log_params(data_params)
    mlflow.log_params(model_params)
    mlflow.set_tag("git_commit", git_commit_hash)
    print("Parameters logged.")

    # --- Подготовка данных для PyTorch ---
    # 1. Преобразуем numpy массивы в torch тензоры
    X_train_tensor = torch.from_numpy(X_train).float()
    y_train_tensor = torch.from_numpy(y_train).float().view(-1, 1)  # (batch_size, 1)
    X_test_tensor = torch.from_numpy(X_test).float()
    y_test_tensor = torch.from_numpy(y_test).float().view(-1, 1)

    # 2. Создаем датасеты
    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

    # 3. Разделяем на обучающую и валидационную выборки вручную
    val_split = model_params["validation_split"]
    dataset_size = len(train_dataset)
    val_size = int(val_split * dataset_size)
    train_size = dataset_size - val_size
    train_subset, val_subset = torch.utils.data.random_split(train_dataset, [train_size, val_size])

    # 4. Создаем загрузчики данных (DataLoader)
    train_loader = DataLoader(dataset=train_subset, batch_size=model_params["batch_size"], shuffle=True)
    val_loader = DataLoader(dataset=val_subset, batch_size=model_params["batch_size"])
    test_loader = DataLoader(dataset=test_dataset, batch_size=model_params["batch_size"])

    # Инициализация модели
    device = torch.device("cpu")  # Требование: работать на CPU

    model = AdaptiveTransformerModel(
        input_dim=n_features,
        embed_dim=model_params["embed_dim"],
        num_heads=model_params["num_heads"],
        ff_dim=model_params["ff_dim"],
        num_layers=model_params["num_layers"],
        output_dim=1,
        dropout=model_params["dropout"],
    ).to(device)

    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=model_params["lr"])

    # Цикл обучения
    for epoch in range(model_params["epochs"]):
        model.train()
        train_losses = []

        for inputs, labels in train_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            train_losses.append(loss.item())

        avg_train_loss = float(np.mean(train_losses))
        mlflow.log_metric("train_mse_loss", avg_train_loss, step=epoch)

        # Валидация
        model.eval()
        val_preds = []
        val_targets = []
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs = inputs.to(device)
                labels = labels.to(device)

                outputs = model(inputs)
                val_preds.append(outputs.detach().cpu().numpy())
                val_targets.append(labels.detach().cpu().numpy())

        val_preds = np.concatenate(val_preds)
        val_targets = np.concatenate(val_targets)

        # Расчет всех метрик
        val_metrics = calculate_metrics(val_targets, val_preds)

        # Логирование метрик валидации
        for name, value in val_metrics.items():
            mlflow.log_metric(f"val_{name}", value, step=epoch)

        print(
            f"Epoch {epoch+1}/{model_params['epochs']} | "
            f"Train Loss: {avg_train_loss:.2f} | "
            f"Val MAE: {val_metrics['mae']:.2f} | "
            f"PHM08: {val_metrics['phm08_score']:.2f}"
        )

    # --- Финальный тест и сохранение ---
    print("\nEvaluating on Test Set...")
    model.eval()
    test_preds = []
    test_targets = []
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            test_preds.append(outputs.detach().cpu().numpy())
            test_targets.append(labels.detach().cpu().numpy())

    test_preds = np.concatenate(test_preds)
    test_targets = np.concatenate(test_targets)

    test_metrics = calculate_metrics(test_targets, test_preds)
    print(f"Test Metrics: {test_metrics}")

    # Логируем финальные метрики с префиксом test_
    for name, value in test_metrics.items():
        mlflow.log_metric(f"test_{name}", value)

    # 1. Сохранение ПОЛНОЙ модели
    mlflow.pytorch.log_model(model, "full_model")

    # 2. Сохранение BACKBONE (State Dict без входного слоя) для Transfer Learning
    # Исключаем веса input_adapter, чтобы клиент мог инициализировать свои
    backbone_state_dict = {k: v for k, v in model.state_dict().items() if "input_adapter" not in k}
    torch.save(backbone_state_dict, "transformer_backbone.pth")
    mlflow.log_artifact("transformer_backbone.pth", artifact_path="transfer_learning_artifacts")

    print("Run Complete. Artifacts logged.")

Starting MLflow run...
Parameters logged.
Epoch 1/7 | Train Loss: 536.98 | Val MAE: 19.17 | PHM08: 8.01
Epoch 2/7 | Train Loss: 504.73 | Val MAE: 19.18 | PHM08: 8.25
Epoch 3/7 | Train Loss: 504.41 | Val MAE: 19.17 | PHM08: 8.04
Epoch 4/7 | Train Loss: 504.67 | Val MAE: 19.16 | PHM08: 8.10
Epoch 5/7 | Train Loss: 80.72 | Val MAE: 4.65 | PHM08: 0.68
Epoch 6/7 | Train Loss: 52.74 | Val MAE: 4.62 | PHM08: 0.68
Epoch 7/7 | Train Loss: 51.14 | Val MAE: 5.06 | PHM08: 0.81

Evaluating on Test Set...
Test Metrics: {'mae': 6.7564287, 'rmse': 8.305587, 'mape': 42.46154427528381, 'phm08_score': 1.2659148214699745}


2025/12/21 14:19:54 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/12/21 14:19:57 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Run Complete. Artifacts logged.
🏃 View run grandiose-skunk-663 at: http://213.21.252.250:5000/#/experiments/3/runs/f3f721f3dfb142b681e1771ead3493b7
🧪 View experiment at: http://213.21.252.250:5000/#/experiments/3
